In [1]:
import pandas as pd
import csv
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_profiling
from nltk.corpus import stopwords
%matplotlib inline

file_data=pd.read_csv("Womens Clothing E-Commerce Reviews.csv",index_col=False)
data_fields=['Review Text','Rating','Recommended IND','Positive Feedback Count','Division Name','Department Name','Class Name']
mydata=pd.DataFrame(data=file_data,columns=data_fields)

import nltk
nltk.download('stopwords')
nltk.download('punkt')

import string
def text_process(review):
    nopunc=[word for word in review if word not in string.punctuation]
    nopunc=''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

mydata['Review Text'].head(5).apply(text_process)

mydata=mydata.dropna(axis=0,how='any')
selected_rating_data = mydata[(mydata['Rating'] == 1) | (mydata['Rating'] == 5)]
X_review=selected_rating_data['Review Text']
y=selected_rating_data['Rating']

from sklearn.feature_extraction.text import CountVectorizer
bow_transformer=CountVectorizer(analyzer=text_process).fit(X_review)

X_review = bow_transformer.transform(X_review)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_review, y, test_size=0.3, random_state=101)

c:\users\vic\appdata\local\programs\python\python37-32\lib\site-packages\pandas_profiling\plot.py:15: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "c:\users\vic\appdata\local\programs\python\python37-32\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\vic\appdata\local\programs\python\python37-32\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\vic\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\vic\appdata\local\programs\python\python37-32\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    ap

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\VIC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\VIC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from imblearn.over_sampling import SMOTE, ADASYN
X_resampled, y_resampled = SMOTE().fit_sample(X_train, y_train)

In [3]:
import collections
collections.Counter(y_resampled)

Counter({5: 8773, 1: 8773})

In [4]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import svm
clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [5]:
predictSVM=clf.predict(X_test)

print(confusion_matrix(y_test, predictSVM))
print('\n')
print(classification_report(y_test, predictSVM))

[[ 164   87]
 [  70 3684]]


             precision    recall  f1-score   support

          1       0.70      0.65      0.68       251
          5       0.98      0.98      0.98      3754

avg / total       0.96      0.96      0.96      4005



In [6]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(52,52,52),max_iter=500)

mlp.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(52, 52, 52), learning_rate='constant',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [7]:
predictionsANN = mlp.predict(X_test)

print(confusion_matrix(y_test, predictionsANN))
print('\n')
print(classification_report(y_test,predictionsANN))

[[ 155   96]
 [  32 3722]]


             precision    recall  f1-score   support

          1       0.83      0.62      0.71       251
          5       0.97      0.99      0.98      3754

avg / total       0.97      0.97      0.97      4005



In [8]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [9]:
predictNB=nb.predict(X_test)

print(confusion_matrix(y_test, predictNB))
print('\n')
print(classification_report(y_test, predictNB))

[[  86  165]
 [  15 3739]]


             precision    recall  f1-score   support

          1       0.85      0.34      0.49       251
          5       0.96      1.00      0.98      3754

avg / total       0.95      0.96      0.95      4005



In [10]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(penalty='l2',solver='newton-cg',max_iter=50, multi_class='ovr')
LR.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=50, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='newton-cg', tol=0.0001,
          verbose=0, warm_start=False)

In [11]:
predictLR=LR.predict(X_test)

print(confusion_matrix(y_test, predictLR))
print('\n')
print(classification_report(y_test, predictLR))

[[ 166   85]
 [  30 3724]]


             precision    recall  f1-score   support

          1       0.85      0.66      0.74       251
          5       0.98      0.99      0.98      3754

avg / total       0.97      0.97      0.97      4005



In [12]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf, X_review, y, cv=10)
scores

array([0.96182635, 0.96479401, 0.96928839, 0.95955056, 0.96104869,
       0.96104869, 0.96104869, 0.95802099, 0.95352324, 0.96626687])

In [13]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(nb, X_review, y, cv=10)
scores

array([0.95134731, 0.96404494, 0.96179775, 0.95580524, 0.95205993,
       0.96779026, 0.96104869, 0.96626687, 0.95652174, 0.95127436])

In [14]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(LR, X_review, y, cv=10)
scores

array([0.96706587, 0.96928839, 0.97303371, 0.96629213, 0.96479401,
       0.97303371, 0.96254682, 0.96326837, 0.96251874, 0.97001499])

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(mlp, X_review, y, cv=10)

In [ ]:
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import svm
clf = svm.SVC(kernel='linear')
from sklearn.model_selection import train_test_split
for i in range(10):    
    X_train, X_test, y_train, y_test = train_test_split(X_review, y, test_size=0.3, random_state=101)
    X_resampled, y_resampled = SMOTE().fit_sample(X_train, y_train)
    

    clf.fit(X_resampled, y_resampled) 
    predictSVM=clf.predict(X_test)

    print(confusion_matrix(y_test, predictSVM))
    print('\n')
    print(classification_report(y_test, predictSVM))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(13,13,13),max_iter=500)
for i in range(10):  
    X_train, X_test, y_train, y_test = train_test_split(X_review, y, test_size=0.3, random_state=101)
    X_resampled, y_resampled = SMOTE().fit_sample(X_train, y_train)
    
   
    mlp = MLPClassifier(hidden_layer_sizes=(13,13,13),max_iter=500)
    mlp.fit(X_resampled,y_resampled)
    predictionsANN = mlp.predict(X_test)
    
    print(confusion_matrix(y_test, predictionsANN))
    print('\n')
    print(classification_report(y_test, predictionsANN))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
for i in range(10): 
    X_train, X_test, y_train, y_test = train_test_split(X_review, y, test_size=0.3, random_state=101)
    X_resampled, y_resampled = SMOTE().fit_sample(X_train, y_train)
    
    
    nb = MultinomialNB()
    nb.fit(X_resampled, y_resampled)
    predictNB=nb.predict(X_test)
    
    print(confusion_matrix(y_test, predictNB))
    print('\n')
    print(classification_report(y_test, predictNB))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(penalty='l2',solver='newton-cg',max_iter=50, multi_class='ovr')
for i in range(10): 
    X_train, X_test, y_train, y_test = train_test_split(X_review, y, test_size=0.3, random_state=101)
    X_resampled, y_resampled = SMOTE().fit_sample(X_train, y_train)
    
    
    LR = LogisticRegression(penalty='l2',solver='newton-cg',max_iter=50, multi_class='ovr')
    LR.fit(X_resampled, y_resampled)
    predictLR=LR.predict(X_test)
    
    print(confusion_matrix(y_test, predictLR))
    print('\n')
    print(classification_report(y_test, predictLR))

In [ ]:
print(bow_transformer.vocabulary_)

In [ ]:
np.set_printoptions(threshold=np.nan)
X_review.todense()[0]